In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [2]:
## this is needed to display the images
%matplotlib inline

##this is neded since the notebook is stored in the object_dection folder.
sys.path.append("..")

In [3]:
from utils import label_map_util
from utils import visualization_utils as vis_util

In [14]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
model_file=model_name + '.tar.gz'

download_base='http://download.tensorflow.org/models/object_detection/'

path_to_ckpt=model_name + '/frozen_inference_graph.pb'

path_to_labels=os.path.join("data","mscoco_label_map.pbtxt")

num_classes=90

Download model

In [15]:
opener=urllib.request.URLopener()
opener.retrieve(download_base + model_file ,model_file)
tar_file=tarfile.open(model_file)

for file in tar_file.getmembers():
    file_name=os.path.basename(file.name)
    if "frozen_inference_graph.pb" in file_name:
        tar_file.extract(file,os.getcwd())

Load a (frozen) tensroflow model into memory

In [20]:
detection_graph=tf.Graph()
with detection_graph.as_default():
    od_graph_def=tf.GraphDef()
    with tf.gfile.GFile(path_to_ckpt,'rb') as fid:
        serialized_graph=fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def,name='')

loading label map

In [21]:
label_map = label_map_util.load_labelmap(path_to_labels)
categoris=label_map_util.convert_label_map_to_categories(label_map,max_num_classes=num_classes)
category_index = label_map_util.create_category_index(categoris)

Helper code

In [22]:
def load_image_into_numpy_array(image):
    (im_width,im_height)=image.size
    return np.array(image.getdata()).reshape((im_height,im_width,3)).astype(np.uint8)

Dection

In [23]:
path_to_test_images_dir='test_images'
test_images_path=[os.path.join(path_to_test_images_dir,'image{}.jpg'.format(i)) for i in range(1,3)]

image_size=(12,8)

In [24]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for image_path in test_images_path:
            image=Image.open(image_path)
            
            image_np=load_image_into_numpy_array(image)
            
            image_np_expanded=np.expand_dims(image_np,axis=0)
            image_tensor=detection_graph.get_tensor_by_name("image_tensor:0")
            
            boxes=detection_graph.get_tensor_by_name("detection_boxes:0")
            
            scores=detection_graph.get_tensor_by_name("detection_score:0")
            
            classes=detection_graph.get_tensor_by_name("detection_classes:0")
            
            num_detection=detection_graph.get_tensor_by_name("num_detection:0")
            
            (boxes,scores,classes,num_detection)=sess.run([boxes,scores,classes,num_detection],
                                                    feed_dict={image_tensor:image_np_expanded})
            
            
            vis_util.visualize_boxes_and_labels_on_image_array(image_np,
                                                              np.squeeze(boxes),
                                                              np.squeeze(classes).astype(np.int32),
                                                              np.squeeze(scores),
                                                              category_index,
                                                              use_normalized_coordinates=True,
                                                              line_thickness=8)
            plt.figure(figsize=image_size)
            plt.imshow(image_np)

KeyError: "The name 'detection_score:0' refers to a Tensor which does not exist. The operation, 'detection_score', does not exist in the graph."